In [1]:

import sys
import os
sys.path.append(os.path.abspath('..'))
from scipy.constants import N_A
from scipy.constants import physical_constants
import numpy as np 
from tools.component_tools import Component
from tools.component_tools import Fluid
from tools.component_tools import Membrane , Geometry 
from tools.materials import Flibe 
import tools.correlations as corr
import tools.materials as materials
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
from scipy.optimize import fsolve
import matplotlib.lines as mlines
from tools.component_tools import BreedingBlanket
from scipy.constants import R

T=973.15
L=10
d_hyd=25.4E-3

mat=materials.Flibe(T)

rho_flibe=2415.6-0.49072*T
n_pipes=2000
n_pipes_HX=3000
mu_flibe=1.16E-4*np.exp(3755/T)
k_H=8.27E-9*np.exp(4.27E-3*T)
D_flibe=9.3E-7*np.exp((-42E3)/(R*T))
K_S_S=953*np.exp((-10.7E3)/(R*T))*1E-3
D_S=7.43E-7*np.exp((-44.1E3)/(R*T))

flibe=Fluid(T=T, Solubility=k_H, MS=True,D=D_flibe, d_Hyd=d_hyd ,mu=mu_flibe,rho=rho_flibe,U0=1,k=mat.k,
    cp=mat.cp)
BB=BreedingBlanket(Q=0.625E9,TBR=1.08,T_in=800,T_out=900,c_in=0, fluid=flibe)
BB.get_flowrate()
BB.get_cout()
c0=BB.c_out
U0=BB.m_coolant/(rho_flibe*np.pi*d_hyd**2/4*n_pipes)
err=1
tol=1E-6
c_in=c0
i=0
geometry=Geometry(D=d_hyd,L=L,thick=0.25E-3)
flibe=Fluid(T=T, Solubility=k_H, MS=True,D=D_flibe, d_Hyd=d_hyd ,mu=mu_flibe,rho=rho_flibe,U0=U0,k=mat.k,
    cp=mat.cp)
Steel = Membrane( T=T,
        D=D_S,
        thick=0.25E-3,
        K_S=K_S_S,
        k_d=1E6,
        k_r=1E6,k=21)
mat_HX=materials.Flibe((BB.T_in+BB.T_out)/2)
d_hyd_HX=20E-3
U0_HX=BB.m_coolant/(mat_HX.rho*np.pi*d_hyd_HX**2/4*n_pipes_HX)
flibe_HX=Fluid(T=(BB.T_in+BB.T_out)/2, Solubility=mat_HX.Solubility, MS=True,D=mat_HX.D, d_Hyd=d_hyd_HX ,mu=mat_HX.mu,U0=U0_HX,rho=mat_HX.rho,k=mat.k,cp=mat_HX.cp)
geometry_HX=Geometry(D=d_hyd_HX,L=L,thick=0.25E-3)
HX = Component(c_in=c_in,geometry=geometry_HX, fluid=flibe_HX, membrane=Steel)
PAV_geometry=Geometry(D=d_hyd,L=L,thick=0.25E-3)
PAV = Component(c_in=c_in,geometry=PAV_geometry, fluid=flibe, membrane=Steel) 
HX.get_global_HX_coeff(0)
circuit_HX=HX.split_HX(N=41,T_in_hot=BB.T_out,T_out_hot=BB.T_in,T_in_cold=581,T_out_cold=800,U=HX.U,Q=BB.Q/n_pipes_HX)

circuit_HX.solve_circuit()
circuit_HX.get_eff_circuit()
print(circuit_HX.eff)

101.48023216563239
104.45543916986956
107.43063470818802
110.40581970760293
113.38099499781435
116.35616132365375
119.33131935566293
122.30646969913235
125.28161290184909
128.25674946076794
131.23187982777563
134.2070044146825
137.18212359757368
140.1572377205991
143.13234709929134
146.10745202348278
149.08255275986622
152.05764955426454
155.03274263362565
158.007832207807
160.98291847115084
163.95800160389294
166.93308177341893
169.908159135392
172.88323383475765
175.8583060066533
178.83337577722514
181.8084432643575
184.78350857834872
187.7585718225003
190.73363309367335
193.70869248277194
196.68375007520726
199.65880595129235
202.63386018663056
205.60891285244256
208.58396401588973
211.5590137403544
214.53406208570289
217.50910910852636
0
c_in: 0.0002940942375326284
geometry is a <class 'tools.component_tools.Geometry'> class, printing its variables:
    L: 10
    D: 0.02
    thick: 0.00025
    n_pipes: 1
eff: None
n_pipes: (1,)
fluid is a <class 'tools.component_tools.Fluid'> class